In [1]:
import tensorflow as tf
import json
import pandas as pd
import numpy as np
import random
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalAveragePooling1D, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [3]:
# Load the JSON file into a Python object
with open('content.json') as content:
    data1 = json.load(content)

In [4]:
#getting all data to lists
inputs = []
tags = []
responses={}
# Iterate through each intent and add its inputs and tags to the lists
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
        inputs.append(lines)
        tags.append(intent['tag'])

In [ ]:
# Create a DataFrame from the lists of inputs and tags
data = pd.DataFrame({'inputs': inputs, 
                   'tags': tags})


In [ ]:
data

In [ ]:
#Preprocessing
#Remove punctuation from the input text
import string
data['inputs'] = data['inputs'].apply(lambda word:[letter.lower() for letter in word if letter not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda word: ''.join(word))
print(data)

In [ ]:
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(df['inputs'])
train = tokenizer.texts_to_sequences(df['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(df['tags'])


In [ ]:
input_shape = x_train.shape[1]
print(input_shape)

In [ ]:
# Define vocabulary
vocabulary = len(tokenizer.word_index)
print("Number of unique words: ", vocabulary)

output_length = le.classes_.shape[0]
print("Output length: ", output_length)

In [ ]:
# Define the model 
i = Input(shape=input_shape)
x = Embedding(vocabulary+1, 10)(i)
x = LSTM(10, return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length, activation="softmax")(x)

model = Model(inputs=i, outputs=x)

In [ ]:
# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
#train the model
train = model.fit(x_train, y_train, epochs=200)

In [ ]:
plt.plot(train.history['accuracy'], label='Training set accuracy')
plt.plot(train.history['loss'], label='Training set loss')
plt.title('Training accuracy and loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy/Loss')
plt.legend()
plt.show()

In [ ]:
#Testing
import random
import string
while True:
    texts_p = []
    prediction_input = input('You: ')

    # Removing punctuation and converting to lowercase
    prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input =''.join(prediction_input)
    texts_p.append(prediction_input)

    # Tokenizing and padding prediction_input
    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)

    # Getting output from model
    output = model.predict(prediction_input)
    output = output.argmax()

    # Finding the right tag and predicting response
    response_tag = le.inverse_transform([output])[0]
    print("Going Merry: ",random.choice(responses[response_tag]))
    if response_tag == "goodbye":
        break